In [1]:
%pip install transformers trl datasets openai unsloth langchain langchain-community langchain-chroma pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opentelemetry-proto to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.3 MB/s eta 0:00:0

In [1]:
from google.colab import drive

In [2]:
# Mount the Google Drive at /content/drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import torch
from huggingface_hub import HfFolder
from datasets import config
max_seq_length = 4096*2 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.9: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [5]:
model = model.to('cuda')

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [7]:
ds = load_dataset("somosnlp-hackathon-2022/Axolotl-Spanish-Nahuatl", split="train")

README.md:   0%|          | 0.00/2.77k [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/6.45M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20028 [00:00<?, ? examples/s]

In [8]:
promptspnh = """Traduce esto al Nahuatl: {} Nahuatl: {}"""
promptnhsp = """Traduce esto al Español: {} Español: {}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formating_input_nh(examples):
  spanish = examples["sp"]
  nahuatl = examples["nah"]
  texts = []
  for sp, nah in zip(spanish, nahuatl):
    text = promptspnh.format(sp, nah) + EOS_TOKEN
    texts.append((text))
  return {"text": texts}

datasetspnh = ds.map(formating_input_nh, batched=True)

def formating_input_sp(examples):
  spanish = examples["sp"]
  nahuatl = examples["nah"]
  texts = []
  for sp, nah in zip(spanish, nahuatl):
    text = promptnhsp.format(nah,sp) + EOS_TOKEN
    texts.append((text))
  return {"text": texts}

datasetnhsp = ds.map(formating_input_sp, batched=True)

Map:   0%|          | 0/20028 [00:00<?, ? examples/s]

Map:   0%|          | 0/20028 [00:00<?, ? examples/s]

In [9]:
datasetnhsp[0]

{'Unnamed: 0': 1,
 'sp': 'Y así, cuando hizo su ofrenda de fuego, se sienta delante de los demás y una persona se queda junto a él.',
 'nah': 'Auh in ye yuhqui in on tlenamacac niman ye ic teixpan on motlalia ce tlacatl itech mocaua.',
 'var': None,
 'src': 'Vida económica de Tenochtitlan',
 'text': 'Traduce esto al Español: Auh in ye yuhqui in on tlenamacac niman ye ic teixpan on motlalia ce tlacatl itech mocaua. Español: Y así, cuando hizo su ofrenda de fuego, se sienta delante de los demás y una persona se queda junto a él.<|eot_id|>'}

In [10]:
trainer1 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasetspnh,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

trainer2 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = datasetnhsp,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,

        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/20028 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Map (num_proc=2):   0%|          | 0/20028 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [11]:
for i in range(15):

  trainer1.train()
  trainer2.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,028 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,4.363300
2,4.754000
3,3.854700
4,4.595600
5,4.200500
6,4.149900
7,4.111500
8,3.765200
9,3.592300
10,3.566000


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,028 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,3.727500
2,3.935200
3,3.437100
4,3.546200
5,3.409100
6,3.442700
7,3.393800
8,3.415700
9,3.146200
10,3.317600


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,028 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 24,313,856


...

Step,Training Loss
1,0.182600
2,0.212500
3,0.131000
4,0.143800
5,0.115500
6,0.182500
7,0.087600
8,0.084300
9,0.099900
10,0.128900


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,028 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.236600
2,0.319900
3,0.159800
4,0.177800
5,0.139200
6,0.260200
7,0.116500
8,0.108400
9,0.099700
10,0.099700


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 20,028 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,0.190300
2,0.248400
3,0.122000
4,0.138300
5,0.099900
6,0.167900
7,0.105700
8,0.080900
9,0.103800
10,0.135100


In [12]:
FastLanguageModel.for_inference(model)
inputs = tokenizer("""Explica el siguiente documento en espanol: Kejatsa:
Tlanauatili tlen ika nopa tlajtlanijketl tlen kinextia para kichijtok nopa tlamantli tlen kiijtoua uan ya kichijtok nopa tlaxtlauili tlen kinamiki, kiselis nopa licencia de conducir tipo Motorist. Español:
""", return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Explica el siguiente documento en espanol: Kejatsa:\nTlanauatili tlen ika nopa tlajtlanijketl tlen kinextia para kichijtok nopa tlamantli tlen kiijtoua uan ya kichijtok nopa tlaxtlauili tlen kinamiki, kiselis nopa licencia de conducir tipo Motorist. Español:\nPara este tipo de transporte los que tienen esta licencia pueden conducirlos en cualquier vehículo, no solo en los mencionados en la letra A.<|eot_id|>']

In [13]:
FastLanguageModel.for_inference(model)
inputs = tokenizer("Traduce al español: nokoltsin itoka Nick Español: ", return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Traduce al español: nokoltsin itoka Nick Español: 1\tcalabaza partida en pedazos triangulares Español: 2\tTomahito Panza Español: 3\tManuel Chahuel Español: 4\tXochitl Español: 5\tAjacachcauyotl Español: 6\tItoca Niman Niquitohua Español: 7\tPinauhua Español: 8\tTmaltzontli Español: 9\tXochitl Españvalta<|eot_id|>']

In [14]:
model.save_pretrained_merged("/content/drive/MyDrive/model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.97 out of 12.67 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 21.45it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00002-of-00002.bin...
Done.


## **Langchain Workflow**

In [36]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import pipeline
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [37]:
os.makedirs("docs", exist_ok=True)
files = [
    "https://www.aguascalientes.gob.mx/ssp/documentos/RequisitosLicencias.pdf",
    "https://portalanterior.ine.mx/archivos2/portal/credencial/pdf-credencial/pasos-en-el-modulo-INE2014.pdf",
]
for url in files:
    file_path = os.path.join("docs", url.rpartition("/")[2])
    urlretrieve(url, file_path)

In [38]:
loader = PyPDFDirectoryLoader("/content/docs")

docs_before_split = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap  = 50,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(metadata={'source': '/content/docs/RequisitosLicencias.pdf', 'page': 0}, page_content='449 910 2055 www.aguascalientes.gob.mx Av. Aguascalientes Ote. S/N, \nCol. Ex-ejido Ojocaliente, C.P. 20190 \n \nREQUISITOS PARA TRÁMITE DE RENOVACIÓN DE LICENCIA DE CONDUCIR \n \n1. Exhibir licencia de conducir del Estado de Aguascalientes. \n2. Identificación oficial vigente con fotografía (original). \n3. Comprobante de domicilio reciente del Estado de Aguascalientes (original o en versión digital). \n4. CURP. \n \na) Si la identificación oficial tiene registro de domicilio de otra Entidad Federativa, el interesado debe \npresentar algún documento que acredite su residencia en Aguascalientes. (Copia simple de un')

In [39]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs])//len(docs)
avg_char_before_split = avg_doc_length(docs_before_split)
avg_char_after_split = avg_doc_length(docs_after_split)

print(f'Before split, there were {len(docs_before_split)} documents loaded, with average characters equal to {avg_char_before_split}.')
print(f'After split, there were {len(docs_after_split)} documents (chunks), with average characters equal to {avg_char_after_split} (average chunk length).')

Before split, there were 17 documents loaded, with average characters equal to 856.
After split, there were 28 documents (chunks), with average characters equal to 515 (average chunk length).


In [40]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [41]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 3.22735719e-02 -1.50728365e-02  3.14782262e-02  4.76160757e-02
  6.09926730e-02 -5.53977527e-02 -1.17234131e-02  1.78370457e-02
  2.12196484e-02  5.14150877e-03 -4.86333594e-02 -1.14055671e-01
 -4.62390408e-02 -1.60591099e-02 -2.41427757e-02 -1.25350589e-02
  3.68407159e-03  8.17795470e-02  2.78700218e-02  1.19262720e-02
  9.37364176e-02  1.49804482e-03 -2.66711563e-02 -1.23039586e-02
 -4.17536031e-03 -1.40676415e-02 -3.62438755e-03  9.64487623e-03
 -2.63472274e-03 -1.53009027e-01 -6.32174909e-02 -2.65284274e-02
 -3.48875113e-02 -5.88622950e-02 -2.56217830e-02 -2.76232716e-02
 -6.12587947e-03  5.98237887e-02  1.68223623e-02 -1.68920085e-02
 -1.77821871e-02  3.17787603e-02 -6.94586895e-03  1.82013903e-02
  7.20225798e-04 -6.06085919e-03  3.41729932e-02  2.09051254e-03
  1.14556048e-02 -7.11720670e-03  2.32802515e-04 -3.73685593e-03
  1.57009512e-02  5.07216118e-02 -5.54080792e-02  4.38768193e-02
  5.93420640e-02 -3.55764516e-02  6.48812056e-02 -4

In [42]:
vectorstore = Chroma.from_documents(docs_after_split, huggingface_embeddings)

In [ ]:
query = """Que documentos llevo a mi tramite de credencial INE"""
# Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
print(relevant_documents[0].page_content)

There are 4 documents retrieved which are relevant to the query. Display the first one:

Derechos Reservados © Instituto Nacional Electoral, 2014.
Conoce el trámite en el Módulo  
para obtener tu Credencial para Votar
Si tienes una Cita te indicarán el escritorio 
de atención que te corresponde, si no la 
programaste, te asignarán un turno.
1 Al entrar al Módulo te solicitarán tus tres 
documentos: 
y te preguntarán qué trámite realizarás:
• Inscripción, cambio de domicilio, corrección de 
datos personales o en dirección; Reposición o 
Reimpresión; Reincorporación, Reemplazo por 
pérdida de vigencia.
1) Documento que acredite  
tu nacionalidad
2) Comprobante de domicilio
3) Identificación con foto
Sin ellos no podrás realizar ningún trámite.


In [46]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
from openai import OpenAI

base_url = "https://api.aimlapi.com/v1"
api_key = ""

api = OpenAI(api_key=api_key, base_url=base_url)

'Que documentos llevo a mi tramite de credencial INE\nPara tramitar la credencial de funcionario de la INE, es necesario presentar los siguientes documentos:\n- Documento de identidad (DNI)\n- Documento de carácter que acredite la naturaleza de tu función laboral (permiso de trabajo, contrato laboral, etc.)\n- Documento que acredite la relación laboral con la institución (tarjeta de trabajo, contrato de trabajo, etc.)\n\n**Nota:** Los requisitos pueden variar dependiendo de la situación específica de cada funcionario. Es recomendable consultar con la institución o con el personal de la INE para obtener información actualizada y precisa.\n\n**Documentos adicionales**\n\n- Documento de nacimiento (si no se ha presentado con el DNI)\n- Documento que acredite la experiencia laboral (certificado de antecedentes, certificado de experiencia laboral, etc.)\n- Documentos que acrediten la naturaleza de tu función laboral (permiso de trabajo, contrato laboral, etc.)\n\n**Requisitos específicos**\

In [ ]:
query = "Que documentos necesito para tramitar mi INE"
docs = vectorstore.similarity_search(query)
context = ""
for document in docs:
  context += document.page_content

In [ ]:
system_prompt = "Responde las preguntas en base al contexto dado"

user_prompt = "Que documentos necesito para tramitar mi INE" + context

completion = api.chat.completions.create(
      model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt},
      ],
      temperature=0.7,
      max_tokens=2048,
  )

In [ ]:
system_prompt = """Generame 25 preguntas en formato JSON sobre el texto compartido. Que se vea asi:
                    JSON:
                    {
                      {
                        "question": "Pregunta 1",
                        "answer": "Respuesta 1"
                      },
                      {
                        "question": "Pregunta 2",
                        "answer": "Respuesta 2"
                      },
                      ...
                      }
                    }
                    """


user_prompt = """Conoce el trámite en el Módulo
para obtener tu Credencial para Votar
Si tienes una Cita te indicarán el escritorio
de atención que te corresponde, si no la
programaste, te asignarán un turno.
1 Al entrar al Módulo te solicitarán tus tres
documentos:
y te preguntarán qué trámite realizarás:
• Inscripción, cambio de domicilio, corrección de
datos personales o en dirección; Reposición o
Reimpresión; Reincorporación, Reemplazo por
pérdida de vigencia.
1) Documento que acredite
tu nacionalidad
2) Comprobante de domicilio
3) Identificación con foto
Sin ellos no podrás realizar ningún trámite.
Deberán ser originales, sin tachaduras ni
enmendaduras.
Revisa los medios de identificiación autorizados
en la página www.ine.mx
Derechos Reservados © Instituto Nacional Electoral, 2014.
Conoce el trámite en el Módulo
2 Capturarán tus datos en el sistema:
• Nombre y domicilio actual
• Todas tus huellas dactilares
• Fotografía digital
• Firma digital
• Se conservará copia
digital de tus
documentos
Capturarán las huellas
de todos tus dedos
Deberás
firmar
en la tableta
digital
Te tomarán una
fotografía digital Digitalizarán
tus documentos
Te mostrarán el formato impreso para revisar
tus datos, es importante que estén correctos
pues así quedarán registrados en tu nueva
credencial.
Deberás poner tu firma y huella autógrafa
(de puño y letra), en los recuadros asignados.
Derechos Reservados © Instituto Nacional Electoral, 2014.
Conoce el trámite en el Módulo
3
4
Finalmente, te entregarán un comprobante con
la fecha a partir de la cual podrás recoger tu
credencial.
Consérvalo, ya que deberás
presentarlo en el módulo
cuando vayas por ella.
A partir de la fecha que indica tu comprobante,
acude al mismo módulo donde realizaste tu
trámite para recogerla.
Identifícate con tus huellas
dactilares y firma de recibido.

I. Introducción
Estimado contribuyente, el Servicio de Administración Tributaria pone a tu
disposición el servicio de Inscripción al RFC con CURP para que realices a través del
Portal del SAT, tu inscripción en el Registro Federal de Contribuyentes (RFC) para lo
cual, es importante que tengas en cuenta lo siguiente:
• Este servicio inicia y concluye a través del Portal del SAT, sin necesidad de
acudir a las oficinas del SAT ni presentar ningún tipo de documento.
• Los requisitos para obtener tu RFC a través de esta opción son:
✓ Tener CURP vigente.
✓ Ser mayor de edad.
✓ Capturar en el formulario de inscripción, los datos de tu domicilio,
correo electrónico y características fiscales.
• Esta opción de inscripción podrá realizarse una sola vez.
• La clave del RFC que se genere en esta inscripción será válida para cualquier
trámite ante el SAT, otras autoridades de gobierno, así como para gestiones
administrativas que realices.
II. Inscripción en el RFC con CURP
En este apartado se orienta a las personas físicas, que desean efectuar su inscripción
en el RFC, a través de la CURP
Paso 1.- Ingresa a la liga siguiente:
https://www.sat.gob.mx/tramites/operacion/28753/obten-tu-rfc-con-la-clave-unicade-registro-de-poblacion-curp
O bien a https://www.sat.gob.mx/personas apartado Trámites del RFCPaso 2.- Del apartado Inscripción al RFC selecciona Obtén tu RFC con la Clave
Única de Registro de Población CURP; coloca el cursor sobre el título y da clic.
Antes de que comiences con la captura de la información, es importante que tomes
en cuenta que:
a) Al inicio de cada pantalla aparecerán instrucciones, que te servirán como
apoyo para la captura de tu información. Léelas cuidadosamente.
b) Existen campos marcados con [*] asterisco, que son obligatorios, por lo que no
pueden quedar vacíos.
c) Algunos de los campos tienen el símbolo [?], en ellos encontrarás ayuda para
la captura.
d) Desliza las barras lateral y horizontal para visualizar todos los campos del
formulario.
Paso 3.- Captura tus datos de identificación, CURP, e introduce el código captcha
(texto de la imagen) respetando mayúsculas y minúsculas, si ésta no es clara,
presiona en el icono para cambiarla y captura nuevamente, da clic en Continuar.
Si requieres verificar los datos de tu CURP puedes ingresar a la
página http://consultas.curp.gob.mx para obtener mayor información.
Paso 4.- Una vez capturada la CURP y los caracteres de la imagen, presiona el
botón Continuar.
De la validación de tu CURP pueden surgir los siguientes mensajes:
a) Si tu CURP ya está asociada a un RFC:
Nota: Para obtener mayor información relacionada con el registro, ingresa a la
siguiente liga : https://www.sat.gob.mx/aplicacion/operacion/31274/consultatu-clave-de-rfc-mediante-curp o puedes acudir a la Administración
Desconcentrada de Servicios al Contribuyente, de tu preferencia, con una
identificación oficial, previa cita.
b) Si tu CURP no es correcta o no está completa:
Nota: Captura nuevamente las letras y números de tu CURP.
c) Si la CURP corresponde a un menor de edad:
Nota: Para la inscripción de un menor de edad, deberá presentarse en la
Administración Desconcentrada de Servicios al Contribuyente de tu preferencia, o
bien, si se encuentra dentro del supuesto siguiente Las personas físicas menores
de edad a partir de los 16 años pueden inscribirse en el RFC, siempre que presten
exclusivamente un servicio personal subordinado (salarios) desde su inscripción
y hasta que tengan 18 años cumplidos, sin que puedan cambiar de régimen
fiscal hasta que cumplan la mayoría de edad; deberán ingresar a la liga siguiente:
https://www.sat.gob.mx/tramites/83843/rfc-con-la-clave-unica-de-registro-depoblacion-(curp),-para-personas-fisicas-menores-de-edad-a-partir-de-16-anos"""

api = OpenAI(api_key=api_key, base_url=base_url)

completion = api.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
        max_tokens=2048,
    )

In [ ]:
import json
preguntas = []

In [ ]:
for i in range(2):

  completion = api.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
        max_tokens=2048,
    )

  json_string = completion.choices[0].message.content.split("```json")[1].split("```")[0]
  data = json.loads(json_string)

  preguntas += [item['question'] for item in data['questions']]

In [ ]:
pairs = []
for pregunta in preguntas:

  docs = vectorstore.similarity_search(pregunta)
  context = ""
  for document in docs:
    context += document.page_content

  system_prompt = "Responde las preguntas en base al contexto dado"

  user_prompt = "Que documentos necesito para tramitar mi INE" + context

  for _ in range(5):

    completion = api.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
        max_tokens=2048,
    )

    pairs.append((pregunta,completion.choices[0].message.content ))



In [16]:
import pandas as pd

columns = ["Pregunta", "Respuesta"]

df = pd.DataFrame(pairs, columns = columns)

NameError: name 'pairs' is not defined

In [ ]:
def generar_pares_respuestas(json_string):
    # Convertir el string JSON a diccionario
    try:
        # Reemplazar corchetes externos por llaves y agregar comillas a "respuestas"
        json_string_corregido = json_string.replace('[{', '{"respuestas": [{').replace('}]', '}]}')
        json_data = json.loads(json_string_corregido)
    except json.JSONDecodeError as e:
        print(f"Error al decodificar JSON: {e}")
        return []

    # Extraer la lista de respuestas del JSON
    respuestas = json_data['respuestas']

    # Crear pares de respuestas donde la primera tenga mejor calificación
    pares_respuestas = []

    # Generar todas las combinaciones posibles de respuestas
    for (j, response_j), (k, response_k) in itertools.product(
        enumerate(respuestas),
        enumerate(respuestas)
    ):
        # Asegurar que j y k sean diferentes
        if j != k:
            # Verificar que la calificación de j sea mayor que la de k
            if response_j['calificacion'] > response_k['calificacion']:
                par = {
                    'response_j': response_j['respuesta'],
                    'response_k': response_k['respuesta']
                }
                pares_respuestas.append(par)

    return pares_respuestas

In [ ]:
dpo_data = []

for j, pregunta in enumerate(df['Pregunta'].unique()):
    respuestas = df.loc[df['Pregunta'] == pregunta, 'Respuesta']

    comptext = "Compara las siguientes respuestas para la siguiente pregunta " + pregunta

    comptext += """y calificalas en un JSON de la siguiente forma. Solo regresa el JSON:
                {
                  "respuestas": [
                    {
                      respuesta: "Transcribe la respuesta... ",
                      calificacion: (Entero del 1 al 10)
                    }
                  ]
                }

                Respuestas:
                """

    for i, respuesta in enumerate(respuestas):

      comptext += f"Respuesta {i}: " + respuesta + "Fin de la respuesta"


    completion = api.chat.completions.create(
        model="anthropic/claude-3.5-sonnet-20241022",
        messages=[
            {"role": "user", "content": comptext}
        ],
        temperature=0.7,
        max_tokens=2048,
    )

    json_data = completion.choices[0].message.content

    pairs = generar_pares_respuestas(json_data)

    for sample in pairs:

      s = [pregunta] + list(sample.values())
      dpo_data.append(s)

    print(f"Pregunta {j} lista")


In [ ]:
columns = ["question", "response_j", "response_k"]

dpodf = pd.DataFrame(dpo_data, columns=columns)

dpodf.to_csv("DPOData.csv")

In [17]:
dpodf = pd.read_csv("DPOData.csv")

In [18]:
dpodf

,Unnamed: 0,question,response_j,response_k
0,0,¿Qué documentos necesitas presentar al entrar ...,Para tramitar tu inscripción en el Instituto N...,"En base al contexto dado, para tramitar tu INE..."
1,1,¿Qué documentos necesitas presentar al entrar ...,Para tramitar tu inscripción en el Instituto N...,Para tramitar tu Inscripción en el Registro Fe...
2,2,¿Qué documentos necesitas presentar al entrar ...,Para tramitar tu inscripción en el Instituto N...,Para tramitar tu Inscripción en el Instituto N...
3,3,¿Qué documentos necesitas presentar al entrar ...,Para tramitar tu inscripción en el Instituto N...,"Para tramitar tu INE, necesitarás los siguient..."
4,4,¿Qué documentos necesitas presentar al entrar ...,Para tramitar tu Inscripción en el Registro Fe...,"En base al contexto dado, para tramitar tu INE..."
...,...,...,...,...
364,364,¿Qué es lo que se digitaliza durante el trámit...,Para tramitar tu Credencial para Votar en el M...,Para tramitar tu INE y obtener tu nueva Creden...
365,365,¿Qué es lo que se digitaliza durante el trámit...,Para tramitar tu Credencial para Votar en el M...,"Para tramitar tu INE en México, necesitarás lo..."
366,366,¿Qué es lo que se digitaliza durante el trámit...,Para tramitar tu Credencial para Votar en el M...,Para tramitar tu Credencial para Votar en el M...
367,367,¿Qué es lo que se digitaliza durante el trámit...,"Para tramitar tu INE en México, necesitarás lo...",Para tramitar tu credencial para votar en el M...


# **DPO Training**

In [19]:
from datasets import Dataset

In [21]:
dataset = Dataset.from_pandas(dpodf)

In [22]:
from unsloth import PatchDPOTrainer
from transformers import TrainingArguments
from trl import DPOTrainer, DPOConfig
from openai import OpenAI
import json
PatchDPOTrainer()

In [27]:
def return_prompt_and_responses(samples):
    return {
        "prompt": [
            "Question: " + question + "\n\nAnswer: "
            for question in samples["question"]
        ],
        "chosen": samples["response_j"],   # rated better than k
        "rejected": samples["response_k"], # rated worse than j
    }

original_columns = dataset.column_names

dataset = dataset.map(
    return_prompt_and_responses,
    batched=True,
    remove_columns=original_columns
)

Map:   0%|          | 0/369 [00:00<?, ? examples/s]

In [28]:
dataset[0]

{'prompt': 'Question: ¿Qué documentos necesitas presentar al entrar al Módulo para obtener la Credencial para Votar?\n\nAnswer: ',
 'chosen': 'Para tramitar tu inscripción en el Instituto Nacional Electoral (INE) y obtener tu Credencial para Votar, necesitarás presentar los siguientes documentos: 1. Documento que acredite tu nacionalidad 2. Comprobante de domicilio 3. Identificación con foto. Es importante mencionar que estos documentos son obligatorios para realizar cualquier trámite en el Módulo del Instituto Nacional Electoral.',
 'rejected': 'En base al contexto dado, para tramitar tu INE (Inscripción Nacional Electrónica) y obtener tu Credencial para Votar, necesitarás los siguientes documentos: 1. Documento que acredite tu nacionalidad 2. Comprobante de domicilio 3. Identificación con foto. Y para completar el proceso de inscripción, necesitarás capturar tus datos de identificación, CURP y introducir el código captcha. Además, al concluir el proceso de inscripción, recibirás un A

In [31]:
training_args = DPOConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
)

In [32]:
dpo_trainer = DPOTrainer(
    model,                 # base model from SFT pipeline
    None,             # typically a copy of the SFT trained base model
    beta=0.1,              # temperature hyperparameter of DPO
    train_dataset=dataset, # dataset prepared above
    tokenizer=tokenizer,   # tokenizer
    args=training_args,    # training arguments e.g. batch size, lr, etc.
)

Extracting prompt from train dataset:   0%|          | 0/369 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/369 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/369 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [34]:
#Final Learning Step
for i in range(5):
  dpo_trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.022000,-1.469337,-8.897181,1.000000,7.427843,-238.938751,-189.361069,-0.698117,-0.615374
2,0.048700,-0.212492,-6.352656,1.000000,6.140165,-213.471359,-206.923019,-0.804255,-0.560422
3,0.078600,-0.308527,-5.750084,1.000000,5.441556,-191.509399,-172.615112,-0.715786,-0.585260
4,0.022500,-2.752520,-6.953089,1.000000,4.200569,-196.323059,-181.671326,-0.717608,-0.740412
5,0.040800,-2.840858,-9.477316,1.000000,6.636459,-236.042267,-206.940018,-0.741616,-0.597131
6,0.121500,0.166765,-5.341249,1.000000,5.508014,-171.131775,-157.597900,-0.753509,-0.591964
7,0.045500,0.127730,-5.970861,1.000000,6.098592,-200.618484,-178.819641,-0.744409,-0.502629
8,0.024200,-2.865307,-9.216248,1.000000,6.350940,-247.895508,-166.360992,-0.719488,-0.720534
9,0.051600,-2.328824,-11.675097,1.000000,9.346272,-281.423096,-203.210434,-0.457846,-0.576690
10,0.023600,-3.208240,-10.233582,1.000000,7.025342,-212.853760,-183.950714,-0.774725,-0.866969


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.016200,-2.894522,-15.045441,1.000000,12.150918,-300.421356,-203.612915,-0.362966,-0.256673
2,0.007100,-1.484254,-9.621683,1.000000,8.137429,-246.161621,-219.640640,-0.450717,-0.250999
3,0.029200,-1.496731,-10.467488,1.000000,8.970757,-238.683426,-184.497147,-0.378309,-0.254562
4,0.001700,-4.000710,-12.958780,1.000000,8.958070,-256.380005,-194.153244,-0.451308,-0.503810
5,0.009100,-4.508354,-14.600325,1.000000,10.091970,-287.272339,-223.614990,-0.461388,-0.339974
6,0.035800,-0.683754,-8.261480,1.000000,7.577726,-200.334076,-166.103073,-0.471761,-0.325224
7,0.003700,-0.585509,-9.221512,1.000000,8.636003,-233.124969,-185.952026,-0.504029,-0.284100
8,0.003200,-2.841709,-12.670696,1.000000,9.828986,-282.439972,-166.125031,-0.477437,-0.480671
9,0.025800,-3.482406,-16.249525,1.000000,12.767118,-327.167358,-214.746262,-0.195755,-0.328694
10,0.005100,-6.187154,-14.796856,1.000000,8.609701,-258.486511,-213.739868,-0.473200,-0.556761


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.000700,-3.804464,-16.582821,1.000000,12.778356,-315.795135,-212.712341,-0.627030,-0.527622
2,0.003900,0.036453,-10.691126,1.000000,10.727579,-256.856049,-204.433578,-0.735905,-0.513860
3,0.011700,-1.388819,-10.823911,1.000000,9.435091,-242.247650,-183.418030,-0.608136,-0.515169
4,0.000300,-4.911754,-14.483888,1.000000,9.572135,-271.631073,-203.263672,-0.739159,-0.694096
5,0.001600,-4.912564,-17.648232,1.000000,12.735667,-317.751404,-227.657074,-0.699027,-0.563820
6,0.065100,1.484038,-8.764614,1.000000,10.248652,-205.365417,-144.425156,-0.720761,-0.534010
7,0.001600,0.000701,-10.189613,1.000000,10.190314,-242.806000,-180.089935,-0.676407,-0.501451
8,0.027000,-2.749217,-15.079298,1.000000,12.330080,-306.526001,-165.200104,-0.730293,-0.622996
9,0.012000,-3.060157,-18.707895,1.000000,15.647739,-351.751099,-210.523773,-0.396080,-0.526178
10,0.000900,-6.465330,-17.254932,1.000000,10.789602,-283.067261,-216.521622,-0.679834,-0.777411


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.000400,-5.035663,-21.112551,1.000000,16.076889,-361.092468,-225.024307,-0.170053,-0.088177
2,0.000800,-0.789013,-13.718674,1.000000,12.929661,-287.131531,-212.688232,-0.279106,-0.114687
3,0.006000,-1.712570,-13.894299,1.000000,12.181728,-272.951538,-186.655548,-0.210479,-0.030302
4,0.000100,-4.961365,-17.522959,1.000000,12.561592,-302.021759,-203.759781,-0.342143,-0.294101
5,0.000800,-5.254195,-21.145411,1.000000,15.891216,-352.723206,-231.073395,-0.293164,-0.177638
6,0.008900,1.051215,-10.266081,1.000000,11.317296,-220.380096,-148.753387,-0.393374,-0.194047
7,0.000100,-0.127955,-13.300482,1.000000,13.172527,-273.914673,-181.376495,-0.318743,-0.142593
8,0.000500,-2.721828,-15.731737,1.000000,13.009909,-313.050385,-164.926208,-0.367165,-0.316796
9,0.006500,-4.460594,-22.838816,1.000000,18.378222,-393.060272,-224.528137,-0.102816,-0.230406
10,0.001300,-6.562431,-19.835468,1.000000,13.273038,-308.872650,-217.492630,-0.371820,-0.486357


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 369 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,rewards / chosen,rewards / rejected,rewards / accuracies,rewards / margins,logps / rejected,logps / chosen,logits / rejected,logits / chosen
1,0.003000,-7.959041,-27.136677,1.000000,19.177635,-421.333740,-254.258087,0.113785,0.204977
2,0.000500,-2.841354,-15.879869,1.000000,13.038515,-308.743469,-233.211655,-0.015895,0.154571
3,0.000700,-3.614743,-16.801300,1.000000,13.186557,-302.021545,-205.677261,0.040979,0.148473
4,0.000000,-7.627022,-21.687170,1.000000,14.060146,-343.663879,-230.416367,-0.028627,0.004155
5,0.000300,-8.050245,-25.930126,1.000000,17.879881,-400.570374,-259.033905,0.008817,0.083522
6,0.004600,0.226740,-12.146191,1.000000,12.372931,-239.181183,-156.998138,-0.070909,0.078212
7,0.000000,-1.516798,-15.322416,1.000000,13.805618,-294.134033,-195.264908,-0.013516,0.124362
8,0.000000,-3.005146,-21.111092,1.000000,18.105946,-366.843933,-167.759369,0.012936,0.006349
9,0.011700,-5.558953,-27.710775,1.000000,22.151825,-441.779877,-235.511749,0.181313,0.067662
10,0.000700,-9.515966,-24.914278,1.000000,15.398313,-359.660706,-247.027985,-0.109674,-0.276124


In [45]:
FastLanguageModel.for_inference(model)

pregunta = "Responde las preguntas en base al contexto dado"

docs = vectorstore.similarity_search("Que documentos necesito para tramitar mi INE")
context = ""
for document in docs:
  context += document.page_content

input = "Que documentos necesito para tramitar mi INE" + context

inputs= tokenizer(input, return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs))

['<|begin_of_text|>Que documentos necesito para tramitar mi INESin ellos no podrás realizar ningún trámite.\nDeberán ser originales, sin tachaduras ni \nenmendaduras.\nRevisa los medios de identificiación autorizados \nen la página www.ine.mxDerechos Reservados © Instituto Nacional Electoral, 2014.\nConoce el trámite en el Módulo  \npara obtener tu Credencial para Votar\nSi tienes una Cita te indicarán el escritorio \nde atención que te corresponde, si no la \nprogramaste, te asignarán un turno.\n1 Al entrar al Módulo te solicitarán tus tres \ndocumentos: \ny te preguntarán qué trámite realizarás:\n• Inscripción, cambio de domicilio, corrección de \ndatos personales o en dirección; Reposición o \nReimpresión; Reincorporación, Reemplazo por \npérdida de vigencia.\n1) Documento que acredite  \ntu nacionalidad\n2) Comprobante de domicilio\n3) Identificación con foto\nSin ellos no podrás realizar ningún trámite.Antes de que comiences con la captura de la información, es importante que tome

In [47]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=1024)
hf = HuggingFacePipeline(pipeline=pipe)

llm = hf

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausa

In [48]:
prompt_template = """Usa la informacion provista para dar una respuesta apropiada:

{context}

Pregunta: {question}

Respuesta:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [49]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [54]:
query =  "Que documentos necesito para tramitar mi INE"
result = retrievalQA.invoke({"query": query})
print(result['result'].split("Respuesta:")[1])
print(llm.invoke("Traduce esto al Nahuatl" + result['result'].split("Respuesta")[1] + " Nahuatl:"))

:
Para tramitar una solicitud en el Módulo de Atención al Ciudadano, necesitarás los siguientes documentos:

1.  Documento que acredite tu nacionalidad (como pasaporte o cédula de ciudadanía);
2.  Comprobante de domicilio (como contrato de compra-venta de una casa, alquiler o otorgamiento);
3.  Identificación con foto (como cédula de ciudadanía, pasaporte o licencia de conducir).

Sin estos documentos, no podrás realizar ningún trámite en el Módulo de Atención al Ciudadano.
Traduce esto al Nahuatl:
Para tramitar una solicitud en el Módulo de Atención al Ciudadano, necesitarás los siguientes documentos:

1.  Documento que acredite tu nacionalidad (como pasaporte o cédula de ciudadanía);
2.  Comprobante de domicilio (como contrato de compra-venta de una casa, alquiler o otorgamiento);
3.  Identificación con foto (como cédula de ciudadanía, pasaporte o licencia de conducir).

Sin estos documentos, no podrás realizar ningún trámite en el Módulo de Atención al Ciudadano. Nahuatl: Para trami

In [55]:
rm -r /content/drive/MyDrive/model

In [56]:
model.save_pretrained_merged("/content/drive/MyDrive/model", tokenizer, save_method = "merged_16bit",)

Unsloth: Will remove a cached repo with size 1.4K


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.72 out of 12.67 RAM for saving.


100%|██████████| 28/28 [00:01<00:00, 27.17it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/MyDrive/model/pytorch_model-00002-of-00002.bin...
Done.


In [60]:
model.push_to_hub_merged("RenZaijay/AjakatlLlama3.2-3B", tokenizer, token="")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.48 out of 12.67 RAM for saving.


100%|██████████| 28/28 [00:45<00:00,  1.63s/it]


Unsloth: Saving tokenizer...

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/RenZaijay/AjakatlLlama3.2-3B
